In [ ]:
import cv2
import numpy as np
from datetime import datetime
from skimage.metrics import structural_similarity as ssim

class TamperDetection:
    def __init__(self, input_size=500, show_result=True, similarity_thresh=0.6, multichannel=True):
        self.input_size = input_size
        self.similarity_thresh = similarity_thresh
        self.multichannel = multichannel
        self.show_result = show_result

    def detect_tampering(self, real_frame, fake_frame):
        # Resize frames
        real_frame_resized = cv2.resize(real_frame, (self.input_size, self.input_size))
        fake_frame_resized = cv2.resize(fake_frame, (self.input_size, self.input_size))

        # Convert to grayscale for SSIM calculation
        real_gray = cv2.cvtColor(real_frame_resized, cv2.COLOR_BGR2GRAY)
        fake_gray = cv2.cvtColor(fake_frame_resized, cv2.COLOR_BGR2GRAY)

        # Compute SSIM and difference map
        score, diff = ssim(real_gray, fake_gray, full=True)
        diff = (diff * 255).astype("uint8")

        # Threshold the difference map
        _, thresh = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)

        # Find contours of tampered regions
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        tampered_detected = False

        for contour in contours:
            if cv2.contourArea(contour) > 50:  
                tampered_detected = True
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(fake_frame_resized, (x, y), (x + w, y + h), (0, 0, 255), 2)

        # Show results if required
        if self.show_result:
            status = "TAMPERING DETECTED" if tampered_detected else "NO TAMPERING DETECTED"
            color = (0, 0, 255) if tampered_detected else (0, 255, 0)
            cv2.putText(fake_frame_resized, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

            cv2.imshow("Tampered Video", fake_frame_resized)
            cv2.imshow("Difference Map", diff)
            cv2.imshow("Threshold Map", thresh)
            cv2.waitKey(1)

        return tampered_detected


def process_videos(real_video_path, fake_video_path):
    detector = TamperDetection(show_result=True)
    real_cap = cv2.VideoCapture(real_video_path)
    fake_cap = cv2.VideoCapture(fake_video_path)

    frame_index = 0

    while real_cap.isOpened() and fake_cap.isOpened():
        real_ret, real_frame = real_cap.read()
        fake_ret, fake_frame = fake_cap.read()

        if not real_ret or not fake_ret:
            break

        frame_index += 1
        print(f"Processing Frame {frame_index}...")

        tampered = detector.detect_tampering(real_frame, fake_frame)

        if tampered:
            print(f"[INFO] Tampering detected in frame {frame_index} at {datetime.now()}")

    real_cap.release()
    fake_cap.release()
    cv2.destroyAllWindows()


#  real and fake videos
real_video_path = r"D:\MINOR_PROJECT\New folder\tamper-detection-main\videos\Clone_Auth1.mp4"
fake_video_path = r"New folder\tamper-detection-main\videos\clone1.mp4"

# Start processing
process_videos(real_video_path, fake_video_path)
